In [9]:
import os
import pandas as pd
import numpy as np
from common import sessions, sessfile, load_metadata
root = '/Users/lukearend/phd/kfx/data/mnt/Ketamine'

In [17]:
from tqdm import tqdm

metadata = {}
missing = '/Users/lukearend/phd/kfx/ref/2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0.imec1.lf.meta'
for region in ['hpc', 'pfc']:
    metadata[region] = {}
    for signal in ['lfp', 'ap']:
        df = []
        for i, sess in tqdm(sessions.iterrows(), total=36):
            metafile = sessfile(sess.path, region, signal, 'meta')
            path = os.path.join(root, metafile)
            try:
                md = pd.Series(load_metadata(path))
            except FileNotFoundError:
                md = pd.Series(load_metadata(missing))
            md = md[[
                'fileCreateTime', 'fileSizeBytes', 'fileTimeSecs', 'firstSample',
                'imAiRangeMax', 'imAiRangeMin', 
                'imMaxInt', 'imSampRate', 'nSavedChans', 'snsApLfSy', 'snsSaveChanSubset',
                'imroTbl', 'snsChanMap',
            ]]
            df.append(md)
        metadata[region][signal] = pd.DataFrame(df, index=sessions.index)

100%|███████████████████████████████████████████| 36/36 [00:02<00:00, 15.60it/s]


In [37]:
for signal in ['lfp']:
    for region in ['pfc', 'hpc']:
        for recid, md in metadata[region][signal].iterrows():
            chanmap = md.snsChanMap
            chans = chanmap.strip('()').split(')(')[1:]
            assert len(chans) == 385
            assert chans[384] == 'SY0;768:768'
            if signal == 'ap':
                assert chans[0] == 'AP0;0:0'
                assert chans[383] == 'AP383;383:383'
            if signal == 'lfp':
                assert chans[0] == 'LF0;384:384'
                assert chans[383] == 'LF383;767:767'